# Course project - Machine Learning on Big Data 
***
In the following, we perform various implementations of gradient descent from [An overview of gradient descent optimization algorithms](https://arxiv.org/pdf/1609.04747.pdf) by Sebastian Ruder (2017)

In this project, our main objective is to solve a **linear regression** problem by minimizing the Mean Square Loss (MSE) between the model predictions $h_{\omega}(x^{(i)})$ and the ground truth $y^{(i)}$ :

$$ \mathcal{J}_{MSE} = \dfrac{1}{n} \sum_{i=1}^{n} ( h_{\omega}(x^{(i)}) - y^{(i)})^{2}$$

where : 
* $\mathcal{D} = \{x^{(i)} ; y^{(i)}\}_{i=1}^{n}$ is the training dataset
* $\omega$ are the weights of the model $\in \mathbb{R}^{1xd}$
* $h_{\omega}$ is a linear approximator : $h_{\omega}(x^{(i)}) = \sum_{i=1}^{d} \omega_i x_{j}^{(i)} = \omega^{T} x_{j}$
#### <font color=blue>  Students : Vincent Gouteux & Louis Monier </font>

In [2]:
#import findspark 
#findspark.init() 
import pyspark
import numpy as np
import time
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
from sklearn.datasets import load_boston, load_diabetes
from IPython.display import Image

In [3]:
sc = pyspark.SparkContext(appName="Course Project")

## 0 - Data preparation
***
* We first used a simple toy dataset to perform linear regression and easily check our implementations. Concretely, we generate n data points sampled from a Gaussian distribution $\mathcal{N}(0,1)$. Noise is added so optimization techniques do not converge too fast.

In [5]:
n = 1000 # number of examples
d = 2 # number of features

In [6]:
bias = np.ones((n,1)) 
X = np.random.normal(loc=0.0, scale=1.0, size=(n,1)) # generate n points from Gaussian distribution
noise = np.random.normal(loc=0.0, scale=1.0, size=(n,1)) # add noise 

y = 5 * X + 2 + noise # simple toy dataset
X = np.hstack((bias, X)) 
data = np.hstack((X, y))
w_star = np.dot(np.linalg.pinv(X), y).T
print("Number of examples : ", n)
print("Number of features : ", d)

Number of examples : 1000
Number of features : 2

## 1 - Warm up with Vanilla Gradient Descent (aka Batch Gradient Descent)

Vanilla Gradient Descent (Vanilla GD) computes the gradient of the loss w.r.t. to the parameters θ for the entire training dataset:

$$ \omega = \omega - \eta . \nabla{\omega} \mathcal{J}(\omega) $$

We mention once and for all (here in the case of Vanilla GD), the derivation of the gardient : 
$$ \nabla{\omega} \mathcal{J}_{MSE} = \dfrac{2}{n} \sum_{i=1}^{n} ( h_{\omega}(x^{(i)}) - y^{(i)}) . x^{(i)} $$
***

In [8]:
w = np.random.randn(1,d)
history_GD = []
eta = 1e-2 # step-size 
nb_iter = 200

print("* Start training..")
for i in range(nb_iter):
    rdd = sc.parallelize(data)
    rdd = rdd.map(lambda x : 2 * ( np.dot(w, x[:-1]) - x[-1] ) * x[:-1])
    rdd = rdd.reduce(lambda x,y : (x+y)) / n
    w -= eta * rdd

    mse = np.linalg.norm(w - w_star)
    history_GD.append(mse)
    
    if (i%20 == 0) :
        print("Iter : [{}/{}] ; MSE = {:.3f}".format(i, nb_iter, mse))
print("* End training..")

* Start training..
Iter : [0/200] ; MSE = 3.368
Iter : [20/200] ; MSE = 2.202
Iter : [40/200] ; MSE = 1.440
Iter : [60/200] ; MSE = 0.941
Iter : [80/200] ; MSE = 0.616
Iter : [100/200] ; MSE = 0.403
Iter : [120/200] ; MSE = 0.264
Iter : [140/200] ; MSE = 0.173
Iter : [160/200] ; MSE = 0.113
Iter : [180/200] ; MSE = 0.074
* End training..

In [9]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(history_GD)
plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title("Vanilla Gradient Descent")
plt.show()

## 2 - Mini-batch Gradient Descent
***
Mini-batch gradient descent performs an update for every mini-batch of n training examples:


$$ \omega = \omega - \eta . \nabla{\omega} \mathcal{J}(\omega ; x^{(i:i+n)} ; y^{(i:i+n)}) $$

In [11]:
# PARTITION WE ARE GOING TO USE FOR ALL GD METHOD

nb_repart = 20 # number of batch  
assert(n > nb_repart)
batch_size = n // nb_repart
print("Batch size :", batch_size)
nb_iter = 10

rdd = sc.parallelize(data)
rdd = rdd.repartition(nb_repart).cache() # divide the data in "nb_repart" number of partitions
rdd = rdd.glom().zipWithIndex() # coalescing all elements within each partition into a list

Batch size : 50

In [12]:
w = np.random.randn(1,d)
history_BGD = []
eta = 1e-2

    
print("* Start training..")
for i in range(nb_iter):
    for j in range(nb_repart):
        batch = rdd.filter(lambda x: x[1] == j)
        batch = batch.flatMap(lambda x: x[0])
        n_in_batch = batch.count()
        if (n_in_batch > 0):
            batch = batch.map(lambda x: 2 * ( np.dot(w, x[:-1]) - x[-1] ) * x[:-1])
            batch = batch.reduce(lambda a,b: (a+b)) / n_in_batch
            w -= eta* batch
            mse = np.linalg.norm(w - w_star)
            history_BGD.append(mse)
        else:
            print("Empty RDD..")

    print("Iter : [{}/{}] ; MSE = {:.3f}".format(i, nb_iter, mse)) 
print("* End training..")

* Start training..
Iter : [0/10] ; MSE = 3.446
Iter : [1/10] ; MSE = 2.240
Iter : [2/10] ; MSE = 1.456
Iter : [3/10] ; MSE = 0.948
Iter : [4/10] ; MSE = 0.618
Iter : [5/10] ; MSE = 0.403
Iter : [6/10] ; MSE = 0.264
Iter : [7/10] ; MSE = 0.174
Iter : [8/10] ; MSE = 0.115
Iter : [9/10] ; MSE = 0.077
* End training..

In [13]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(history_BGD)
plt.xlabel("Number of gradient updates")
plt.ylabel("Loss")
plt.title("Mini-batch Gradient Descent")
plt.show()

## 3 - Stochastic Gradient Descent (SGD)
***
We tried several methods to perform SGD. The first one was to create as many repartitions as points in the dataset. We observed the repartitions were not uniform which resulted in a lot of of empty useless RDDs. Moreover, even if we dealt with empty RDDs, the running time was very long, which is the opposite of what SGD was designed for. 

We then decided to move on and implement SGD using some tricks. This approach does not exploit as much the benefits of the Map Reduce framework but it works properly. As many optimizers in the following are based on SGD, we decided to keep this version to have a stable, well-performing reference and show relevant comparisons. To be as thorough as possible, we also implement all the following gradient descent variants based on classic gradient descent.

We recall the gradient update rule for SGD which consists to uniformaly-at-random select an example from the dataset $\{x^{(i)} ; y^{(i)}\}$ (instead of the entire dataset for Vanilla GD or a subset of the dataset for Batch GD) and evaluate the gradient : 

$$ \omega = \omega - \eta . \nabla{\omega} \mathcal{J}(\omega ; x^{(i)} ; y^{(i)}) $$

In [15]:
w = np.random.randn(1,d)
history_SGD = []
eta = 1e-2
print("Batch size :", batch_size)
nb_iter = 10
print("* Start training..")
for j in range(nb_repart):
    batch = rdd.filter(lambda x: x[1] == j)
    batch = batch.flatMap(lambda x: x[0])
    n_in_batch = batch.count()
    if (n_in_batch > 0):
      for k,r in enumerate(batch.take(n_in_batch)) : 
        x = r[:2]
        y = r[2:]
        grad = 2*x*(np.dot(w,x) - y )
        w = w - eta*grad
        mse = np.linalg.norm(w-w_star)
        history_SGD.append(mse)
    else:
        print("Empty RDD..")

    if (j%4 == 0) :
      print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 
print("* End training..")

Batch size : 50
* Start training..
Iter : [0/20] ; MSE = 1.603
Iter : [4/20] ; MSE = 0.169
Iter : [8/20] ; MSE = 0.116
Iter : [12/20] ; MSE = 0.137
Iter : [16/20] ; MSE = 0.138
* End training..

In [16]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(history_GD, label = 'GD')
plt.plot(history_BGD, label = 'BGD')
plt.plot(history_SGD[:200], label = 'SGD')


plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Summary/comparison of different gradient descent methods')
plt.legend();

Here we sums up pros and cons of basic gradient methods seen above (based on the [article](https://arxiv.org/pdf/1609.04747.pdf)) : 
* Vanilla GD can be very slow and is intractable for datasets that do not fit in memory. Here we used a toy training set so the method works and show good convergence.
* Vanilla GD is redundant computations for large datasets, as it recomputes gradients for similar examples before each parameter update
* SGD does away with this redundancy by performing one update at a time. It is much faster than vanilla GD and can be used online. However, SGD has a much high variance.
* Mini-batch GD takes the best of both worlds. It is a trade-off between vanilla GD and SGD. 

Now, let's review some more sophisticated gradient descent optimization algorithms.

## 4 - Momentum
***
SGD is subject to oscillations during training. Momentum is a method that helps accelerate SGD in the relevant direction and dampens oscillations. The method consists of adding a fraction $\gamma$ of the update vector of the past time step to the current update vector :

$$ v_t = \gamma v_{t-1} + \eta  \nabla{\omega} \mathcal{J}(\omega) $$
$$ \omega = \omega - v_t $$

In [19]:
w = np.zeros((1,d))
history_momentum = []
v_prev = 0. # initialization
w = np.random.randn(1,d)
eta = 1e-2
gamma = 0.8
print("Batch size :", batch_size)   
print("* Start training..")
for j in range(nb_repart):
    batch = rdd.filter(lambda x: x[1] == j)
    batch = batch.flatMap(lambda x: x[0])
    n_in_batch = batch.count()
    if (n_in_batch > 0):
      for k,r in enumerate(batch.take(n_in_batch)) : 
        x = r[:2]
        y = r[2:]
        grad = 2*x*(np.dot(w,x) - y )
        if (k == 0) :
          vlast = 0
          v = eta*grad
        else :
          v = gamma*vlast + eta*grad
          vlast = v
        w = w - v
        if (i%10 == 0 ) :
          print(w)
        mse = np.linalg.norm(w-w_star)
        history_momentum.append(mse)
    else:
        print("Empty RDD..")
    if (j%4 == 0) :
      print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 
print("* End training..")

Batch size : 50
* Start training..
Iter : [0/20] ; MSE = 0.535
Iter : [4/20] ; MSE = 0.084
Iter : [8/20] ; MSE = 0.242
Iter : [12/20] ; MSE = 0.331
Iter : [16/20] ; MSE = 0.165
* End training..

In [20]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')
plt.plot(history_SGD[:200], label = 'SGD')
plt.plot(history_momentum[:200], label = 'SGD Momentum')

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Comparison MSE SGD vs GD Momentum')
plt.legend();

As the gamma parameter seems to play an important role in the momentum gradient descent, we decided to perform hyperparameter tuning to estimate optimal range of values.

In [22]:
### MOMENTUM ### #finding best gamma
history_gamma = []
for j in range (10): 
    gamma = 0.1*j
    w = np.zeros(2)
    step = 0.02 
    print("Gamma = ", gamma)
    for j in range(nb_repart):
        batch = rdd.filter(lambda x: x[1] == j)
        batch = batch.flatMap(lambda x: x[0])
        n_in_batch = batch.count()
        if (n_in_batch > 0):
          for k,r in enumerate(batch.take(n_in_batch)) : 
            x = r[:2]
            y = r[2:]
            grad = 2*x*(np.dot(w,x) - y )
            if (k == 0) :
              vlast = 0
              v = eta*grad
            else :
              v = gamma*vlast + eta*grad
              vlast = v
            w = w - v
            if (i%10 == 0 ) :
              print(w)
            mse = np.linalg.norm(w-w_star)
            history_gamma.append(mse)
        else:
            print("Empty RDD..")
print(len(history_gamma))

Gamma = 0.0
Gamma = 0.1
Gamma = 0.2
Gamma = 0.30000000000000004
Gamma = 0.4
Gamma = 0.5
Gamma = 0.6000000000000001
Gamma = 0.7000000000000001
Gamma = 0.8
Gamma = 0.9
10000

In [23]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_gamma[:200], label = 'Gamma = 0.0')
plt.plot(history_gamma[1000:1200], label = 'Gamma = 0.1')
plt.plot(history_gamma[2000:2200], label = 'Gamma = 0.2')
plt.plot(history_gamma[3000:3200], label = 'Gamma = 0.3')
plt.plot(history_gamma[4000:4200], label = 'Gamma = 0.4')
plt.plot(history_gamma[5000:5200], label = 'Gamma = 0.5')
plt.plot(history_gamma[6000:6200], label = 'Gamma = 0.6')
plt.plot(history_gamma[7000:7200], label = 'Gamma = 0.7')
plt.plot(history_gamma[8000:8200], label = 'Gamma = 0.8')
plt.plot(history_gamma[9000:9200], label = 'Gamma = 0.9')
plt.title('MSE for different gradient descent momentum gammas')
plt.legend()

## 5 -  Nesterov accelerated gradient (NAG)
***
NAG is a way to give our momentum term this kind of prescience. We know that we will use our momentum term γvt−1 to move the parameters $\theta$. Computing $\theta -  \gamma v_{t-1}$ thus gives us an approximation of the next position of the parameters (the gradient is missing for the full update), a rough idea where our parameters are going to be :

$$ v_t = \gamma v_{t-1} + \eta . \nabla{\omega} \mathcal{J}(\omega - \gamma v_{t-1}) $$
$$ \omega = \omega - v_t $$

In [25]:
w = np.zeros((1,d))
history_nesterov = []
v_prev = 0. # initialization
eta = 1e-2
gamma = 0.8
print("Batch size :", batch_size)
print("* Start training..")
for j in range(nb_repart):
    batch = rdd.filter(lambda x: x[1] == j)
    batch = batch.flatMap(lambda x: x[0])
    n_in_batch = batch.count()
    if (n_in_batch > 0):
      for k,r in enumerate(batch.take(n_in_batch)) : 
        x = r[:2]
        y = r[2:]
        if (i == 0) :
          vlast = 0
          grad = 2*x*(np.dot(w,x) - y )
          v = step*grad
        else :
          grad = 2*x*(np.dot(w-v,x) - y )
          v = gamma*vlast + step*grad
          vlast = v
        w = w - v
        if (i%10 == 0 ) :
          print(w)
        mse = np.linalg.norm(w-w_star)
        #print(mse)
        history_nesterov.append(mse)
    else:
        print("Empty RDD..")
    if (j%4 == 0) :
      print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 
print("* End training..")

Batch size : 50
* Start training..
Iter : [0/20] ; MSE = 0.786
Iter : [4/20] ; MSE = 0.254
Iter : [8/20] ; MSE = 0.298
Iter : [12/20] ; MSE = 0.295
Iter : [16/20] ; MSE = 0.244
* End training..

In [26]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_GD, label = 'SGD ')
plt.plot(history_momentum[:200], label = 'SGD Momentum')
plt.plot(history_nesterov[:200], label = 'SGD Nesterov ')

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Comparison MSE SGD vs GD Momentum')
plt.legend();

## 6 - Adagrad
***
Adagrad adapts the learning rate to the parameters, performing larger updates for infrequent and smaller updates for frequent parameters. For this reason, it is well-suited for dealing with sparse data. The gradient update rule is given by the following expression :

$$ \omega_{t+1} = \omega_{t} - \dfrac{\eta}{\sqrt(G_T + \epsilon)} \odot g_t $$

In [28]:
w = np.zeros((1,d))
history_adagrad = []
sum_grad = np.zeros((1,d))
eta = 0.7
gamma = 0.9 # classic value
nb_iter = 200
epsilon = 1e-8
print("Batch size :", batch_size)
print("* Start training..")
for j in range(nb_repart):
    batch = rdd.filter(lambda x: x[1] == j)
    batch = batch.flatMap(lambda x: x[0])
    n_in_batch = batch.count()
    if (n_in_batch > 0):
      for k,r in enumerate(batch.take(n_in_batch)) : 
        x = r[:2]
        y = r[2:]
        grad = 2*x*(np.dot(w,x) - y )
        sum_grad += grad**2
        adjusted_eta = eta / np.sqrt(epsilon + sum_grad)
        w -= np.multiply(adjusted_eta, grad) # element-wise 
    
        mse = np.linalg.norm(w - w_star)
        history_adagrad.append(mse)  
        if (i%10 == 0 ) :
          print(w)
    else:
        print("Empty RDD..")
    if (j%4 == 0) :
      print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 
print("* End training..")

Batch size : 50
* Start training..
Iter : [0/20] ; MSE = 0.355
Iter : [4/20] ; MSE = 0.183
Iter : [8/20] ; MSE = 0.108
Iter : [12/20] ; MSE = 0.154
Iter : [16/20] ; MSE = 0.139
* End training..

In [29]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_GD, label = 'SGD ')
plt.plot(history_momentum[:200], label = 'SGD Momentum')
plt.plot(history_nesterov[:200], label = 'SGD Nesterov ')
plt.plot(history_adagrad[:200], label = 'SGD Adagrad ')

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Comparison')
plt.legend();

Adagrad is working fine, however we noticed that we needed a step much larger than for momentum and Nesterov. Changing the step size $\eta$ makes the comparison between methods harder. However, if we select such value for Adagrad, the curve is smoother and we reach approximately the same convergence.

## 7 -  Adadelta
***
It is an extension of Adagrad that seeks to reduce its aggressive, monotonically decreasing learning rate. Instead of accumulating all past squared gradients, Adadelta restricts the window of accumulated past gradients : the sum of gradients is recursively defined as a decaying average of all past squared gradients. 

$$ \omega_{t+1} = \omega_{t} - \dfrac{RMS[\Delta \omega]_{t+1}}{RMS[g]_{t}} g_t $$
$$ \omega_{t+1} = \omega_{t} + \Delta \omega_t $$

In [32]:
w = np.zeros((1,d))
history_adadelta = []
delta = np.zeros((1,d))
Eg_prev = 0.
Ed_prev = 0.
RMSd = 0.

eta = 3e-3
gamma = 0.9 # classic value
nb_iter = 200
epsilon = 1e-8

print("Batch size :", batch_size)

print("* Start training..")
for j in range(nb_repart):
    batch = rdd.filter(lambda x: x[1] == j)
    batch = batch.flatMap(lambda x: x[0])
    n_in_batch = batch.count()
    if (n_in_batch > 0):
      for k,r in enumerate(batch.take(n_in_batch)) : 
        x = r[:2]
        y = r[2:]
        grad = 2*x*(np.dot(w,x) - y ) /n
        
        Eg = gamma * Eg_prev + (1-gamma) * grad**2
        RMSg = np.sqrt(Eg  + epsilon)

        w -= (RMSd / RMSg) * grad

        ###
        delta -= (eta / RMSg) * grad
        Ed = gamma * Eg_prev + (1-gamma) * delta**2
        RMSd = np.sqrt(Ed  + epsilon)

        Eg_prev = Eg
        Ed_prev = Ed

        mse = np.linalg.norm(w - w_star)
        history_adadelta.append(mse)
    else:
        print("Empty RDD..")
    if (j%4 == 0):
      print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 
print("* End training..")


Batch size : 50
* Start training..
Iter : [0/20] ; MSE = 4.410
Iter : [4/20] ; MSE = 0.133
Iter : [8/20] ; MSE = 0.137
Iter : [12/20] ; MSE = 0.242
Iter : [16/20] ; MSE = 0.187
* End training..

In [33]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_adadelta[:200])

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Adadelta');

## 8 - RMSprop
***
RMSprop as well divides the learning rate by an exponentially decaying average of squared gradients :

$$ E[{g^2}]_t = 0.9 E[{g^2}]_{t-1} + 0.1 g_{t}^2 $$
$$ \omega_{t+1} = \omega_{t} - \dfrac{\eta}{\sqrt{E[g^2]_t + \epsilon}} g_t $$

In [35]:
w = np.zeros((1,d))
history_RMS = []
delta = np.zeros((1,d))
Eg_prev = 0.
Ed_prev = 0.

eta = 3e-2
gamma = 0.9 # classic value
nb_iter = 200
epsilon = 1e-8

print("* Start training..")
for j in range(nb_repart):
  batch = rdd.filter(lambda x: x[1] == j)
  batch = batch.flatMap(lambda x: x[0])
  n_in_batch = batch.count()
  if (n_in_batch > 0):
    for k,r in enumerate(batch.take(n_in_batch)) : 
      x = r[:2]
      y = r[2:]
      grad = 2*x*(np.dot(w,x) - y )/n

      ###
      Eg = gamma * Eg_prev + (1-gamma) * grad**2
      RMSg = np.sqrt(Eg  + epsilon)

      w -= (eta / RMSg) * grad

      Eg_prev = Eg

      mse = np.linalg.norm(w - w_star)
      history_RMS.append(mse)
  if (j%4 == 0): 
    print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 

print("* End training..")

* Start training..
Iter : [0/20] ; MSE = 3.994
Iter : [4/20] ; MSE = 0.262
Iter : [8/20] ; MSE = 0.126
Iter : [12/20] ; MSE = 0.183
Iter : [16/20] ; MSE = 0.151
* End training..

In [36]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_adadelta[:300], label = 'Adadelta')
plt.plot(history_RMS[:300], label = ' RMSProp')

plt.legend()
plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('RMSprop');

## 9 - Adaptive Moment Estimation (Adam)
***
Another method that computes adaptive learning rates for each parameter :

$$ m_t = \beta_1 m_{t-1} + (1 - \beta_1) g_t $$
$$ v_t = \beta_2 v_{t-1} +(1 - \beta_2) g_t^2 $$
$$ \omega_{t+1} = \omega_t - \dfrac{\eta}{\sqrt{\hat{v}_t} + \epsilon} \hat{m}_t $$

In [38]:
w = np.zeros((1,d))
history_adam = []
delta = np.zeros((1,d))
mom_prev = 0.
v_prev = 0.

eta = 5e-3
nb_iter = 200
epsilon = 1e-8
beta1 = 0.9
beta2 = 0.999

print("* Start training..")
for j in range(nb_repart):
  batch = rdd.filter(lambda x: x[1] == j)
  batch = batch.flatMap(lambda x: x[0])
  n_in_batch = batch.count()
  if (n_in_batch > 0):
    for k,r in enumerate(batch.take(n_in_batch)) : 
      x = r[:2]
      y = r[2:]
      grad = 2*x*(np.dot(w,x) - y )/n
      mom = beta1 * mom_prev + (1-beta1) * grad 
      v = beta2 * v_prev + (1-beta2) * grad**2

      avg_mom = mom / (1 - beta1)
      avg_v = v / (1 - beta1)

      w -= (eta / np.sqrt(avg_v)) * avg_mom

      mse = np.linalg.norm(w - w_star)
      history_adam.append(mse)
      
      v_prev = v
      mom_prev = mom 
  if (j%4 == 0 ):
    print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse)) 

print("* End training..")

* Start training..
Iter : [0/20] ; MSE = 2.138
Iter : [4/20] ; MSE = 0.252
Iter : [8/20] ; MSE = 0.155
Iter : [12/20] ; MSE = 0.104
Iter : [16/20] ; MSE = 0.099
* End training..

In [39]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_adam[:300], label = 'Adam')
plt.plot(history_adadelta[:300], label = 'Adadelta')
plt.plot(history_RMS[:300], label = 'RMSProp')

plt.legend()
plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.title('Adam');

## 10 - AdaMax
***
It is a generalization of the $v_t$ update with the $l_{\infty}$ norm :


$$ u_t = \beta_2^{\infty} v_{t-1} +(1 - \beta_2^{\infty}) \mid g_t\mid^{\infty} = max(\beta_2 . v_{t-1}, \mid g_t \mid) $$
$$ \omega_{t+1} = \omega_t - \dfrac{\eta}{u_t} \hat{m}_t $$

In [41]:
w = np.zeros((1,d))
history_adamax = []
delta = np.zeros((1,d))
mom_prev = 0.
u_prev = 0.

eta = 1e-3
nb_iter = 200
epsilon = 1e-8
beta1 = 0.9
beta2 = 0.999
print("* Start training..")
for j in range(nb_repart):
  batch = rdd.filter(lambda x: x[1] == j)
  batch = batch.flatMap(lambda x: x[0])
  n_in_batch = batch.count()
  if (n_in_batch > 0):
    for k,r in enumerate(batch.take(n_in_batch)) : 
      x = r[:2]
      y = r[2:]
      grad = 2*x*(np.dot(w,x) - y )/n

      mom = beta1 * mom_prev + (1-beta1) * grad 
      v = beta2 * v_prev + (1-beta2) * grad**2

      avg_mom = mom / (1 - beta1)
      avg_v = v / (1 - beta1)
      u = np.maximum(beta2 * v_prev, np.abs(grad))

      avg_mom = mom / (1 - beta1)
      avg_v = v / (1 - beta1)
      w -= (eta / u) * avg_mom
      
      v_prev = v
      mom_prev = mom 

      mse = np.linalg.norm(w - w_star)
      history_adamax.append(mse)
  if ( j%4 == 0):
    print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse))
print("* End training..")

* Start training..
Iter : [0/20] ; MSE = 2.912
Iter : [4/20] ; MSE = 0.271
Iter : [8/20] ; MSE = 0.368
Iter : [12/20] ; MSE = 0.085
Iter : [16/20] ; MSE = 0.289
* End training..

In [42]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_adamax[:300], label = 'Adamax')
plt.plot(history_adam[:300], label = 'Adam')
plt.plot(history_adadelta[:300], label = 'Adadelta')
plt.plot(history_RMS[:300], label = 'RMSProp')

plt.legend()

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")


## 11 - Nadam 
***
This is a combination of Adam and Nestorov :

$$ \Delta \omega_t = - \dfrac{RMS[\Delta \omega]_{t-1}}{RMS[g]_t} g_t $$
$$ \omega_{t+1} = \omega_t + \Delta \omega_t $$

In [44]:
w = np.zeros((1,d))
history_nadam = []
delta = np.zeros((1,d))
mom_prev = 0.
u_prev = 0.

eta = 5e-2
nb_iter = 200
epsilon = 1e-8
beta1 = 0.9
beta2 = 0.999
print("* Start training..")
for j in range(nb_repart):
  batch = rdd.filter(lambda x: x[1] == j)
  batch = batch.flatMap(lambda x: x[0])
  n_in_batch = batch.count()
  if (n_in_batch > 0):
    for k,r in enumerate(batch.take(n_in_batch)) : 
      x = r[:2]
      y = r[2:]
      grad = 2*x*(np.dot(w,x) - y )/n

      mom = beta1 * mom_prev + (1-beta1) * grad 
      v = beta2 * v_prev + (1-beta2) * grad**2

      avg_mom = mom / (1 - beta1)
      avg_v = v / (1 - beta1)
      u = np.maximum(beta2 * v_prev, np.abs(grad))

      avg_mom = mom / (1 - beta1)
      avg_v = v / (1 - beta1)
      w -= (eta / (np.sqrt(avg_v)+ epsilon)) * (beta1 * avg_mom + ((1-beta1)* grad)/(1-beta1))
      
      v_prev = v
      mom_prev = mom 

      mse = np.linalg.norm(w - w_star)
      history_nadam.append(mse)
  if (j%4 == 0):
    print("Iter : [{}/{}] ; MSE = {:.3f}".format(j, nb_repart, mse))
print("* End training..")

* Start training..
Iter : [0/20] ; MSE = 0.734
Iter : [4/20] ; MSE = 0.127
Iter : [8/20] ; MSE = 0.229
Iter : [12/20] ; MSE = 0.409
Iter : [16/20] ; MSE = 0.131
* End training..

In [45]:
figure(num=None, figsize=(11, 8), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_nadam[:300], label = 'Nadam')
plt.plot(history_adamax[:300], label = 'Adamax')
plt.plot(history_adam[:300], label = 'Adam')
plt.plot(history_adadelta[:300], label = 'Adadelta')
plt.plot(history_RMS[:300], label = 'RMSProp')

plt.legend()

plt.legend()

plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")


In [46]:
figure(num=None, figsize=(15, 11), dpi=80, facecolor='w', edgecolor='k')

plt.plot(history_GD, label = 'GD')
plt.plot(history_BGD, label = 'BGD')
plt.plot(history_SGD[:200], label = 'SGD')
plt.plot(history_momentum[:200], label = 'Momentum')
plt.plot(history_nesterov[:200], label = 'Nesterov')
plt.plot(history_adagrad[:200], label = 'Adagrad')
plt.plot(history_adadelta[:200], label = 'Adadelta')
plt.plot(history_RMS[:200], label = 'RMS')
plt.plot(history_adam[:200], label = 'Adam')
plt.plot(history_adamax[:200], label = 'Adamax')
plt.plot(history_nadam[:200], label = 'Nadam')


plt.xlabel("Number of gradient updates")
plt.ylabel("Mean Square Error")
plt.legend()